In [8]:
from pathlib import Path
from math import sqrt
import polars as pl
from skrub import TableReport
from plotly import express as px

dataset = "ACSEmployment_binarized"
# base_model = "skrub_logistic"
base_model = "skrub"
audit_budget = 1_000
# n_repetitions = 15
n_repetitions = 5
# n_repetitions = 1
entropy = 12345678
tnr = 1.0
tpr = 1.0
output = Path(f"../generated/stealthiness{n_repetitions}_{dataset}_{base_model}.jsonl")

params = [
    "dataset",
    "model_name",
    # "model_params",
    "strategy",
    "strategy_params",
    "audit_budget",
    "detection_tpr",
    "detection_tnr",
    # "entropy",
]
records = pl.read_ndjson(output)

In [9]:
TableReport(records.select(pl.all().exclude("strategy_params")))

Processing column  21 / 21


,,,,,,,,,,,,,,,,,,,,


In [10]:
records = (
    pl.read_ndjson(output)
    # For now, all model params are None so we can drop them
    .select(pl.all().exclude("model_params"))
    # The auditset hamming is what the auditor measures to detect manipulations
    .with_columns(
        auditset_hamming=1 - pl.col("utility_audit"),
        hidden_demographic_parity=(
            pl.col("demographic_parity_audit")
            - pl.col("demographic_parity_audit_honest")
        ).abs(),
        hidden_absolute_demographic_parity=(
            pl.col("absolute_demographic_parity_audit")
            - pl.col("absolute_demographic_parity_audit_honest")
        ).abs(),
    )
    # Group by all the parameters (model, manipulation and detection params)
    .group_by(params)
    # Compute the average and standart deviation over the differents seeds
    .agg(
        pl.all().mean(),
        pl.all().std().name.suffix("_std"),
    )
    # Compute the "error bars" and convert the struct columns into
    # jsonstring columns for plotly to be happy
    .with_columns(
        pl.col("demographic_parity_audit_std") / sqrt(n_repetitions),
        pl.col("auditset_hamming_std") / sqrt(n_repetitions),
        pl.col("manipulation_hamming_std") / sqrt(n_repetitions),
        pl.col("strategy_params").struct.json_encode(),
    ).sort("strategy")
)

# Preliminary analysis of the audit vs user perf metrics

In [14]:
estimation_variance = (
    pl.read_ndjson(
        f"../generated/estimation_variance{n_repetitions}_{dataset}_{base_model}.jsonl"
    )
    # For now, all model params are None so we can drop them
    .select(pl.all().exclude("model_params"))
    # The auditset hamming is what the auditor measures to detect manipulations
    .with_columns(
        auditset_hamming=1 - pl.col("utility_audit"),
        hidden_demographic_parity=(
            pl.col("demographic_parity_audit")
            - pl.col("demographic_parity_audit_honest")
        ).abs(),
        hidden_absolute_demographic_parity=(
            pl.col("absolute_demographic_parity_audit")
            - pl.col("absolute_demographic_parity_audit_honest")
        ).abs(),
    )
    # Group by all the parameters (model, manipulation and detection params)
    .group_by(params)
    # Compute the average and standart deviation over the differents seeds
    .agg(
        pl.all().mean(),
        pl.all().std().name.suffix("_std"),
    )
    # Compute the "error bars" and convert the struct columns into
    # jsonstring columns for plotly to be happy
    .with_columns(
        pl.col("demographic_parity_audit_honest") / sqrt(n_repetitions),
        params_str=pl.concat_str(pl.col(params).exclude("audit_budget")),
    )
    # Sort because plotly does not
    .sort("audit_budget")
)
fig = px.line(
    estimation_variance,
    x="audit_budget",
    y="demographic_parity_audit",
    color="params_str",
    category_orders=dict(strategy=sorted(records["strategy"].sort().unique())),
    # error_x="auditset_hamming_std",
    error_y="demographic_parity_audit_std",
    height=800,
)
fig.add_hline(
    estimation_variance["demographic_parity_user"].mean(),
    line_dash="dash",
    annotation_text="demographic parity user",
)
fig.update_traces(marker_size=20)
fig.show()

# How much can a platform be unfair and how far does it have to move away from its OG model?

In [20]:
fig = px.scatter(
    records,
    x="auditset_hamming",
    y="demographic_parity_audit",
    color="strategy",
    category_orders=dict(strategy=sorted(records["strategy"].sort().unique())),
    error_x="auditset_hamming_std",
    error_y="demographic_parity_audit_std",
    hover_data=["strategy_params"],
    height=800,
)
fig.update_traces(marker_size=20)
fig.show()

: 

: 

In [21]:
fig = px.scatter(
    records,
    x="manipulation_hamming",
    y="demographic_parity_audit",
    color="strategy",
    category_orders=dict(strategy=sorted(records["strategy"].sort().unique())),
    error_x="manipulation_hamming_std",
    error_y="demographic_parity_audit_std",
    hover_data=["strategy_params"],
    height=800,
)
fig.update_traces(marker_size=20)
fig.show()

: 

: 

In [22]:
fig = px.scatter(
    records.with_columns(),
    x="auditset_hamming",
    y="hidden_demographic_parity",
    color="strategy",
    category_orders=dict(strategy=sorted(records["strategy"].sort().unique())),
    # error_x="auditset_hamming_std",
    # error_y="demographic_parity_audit_std",
    hover_data=["strategy_params"],
    height=800,
)
fig.update_traces(marker_size=20)
fig.show()

: 

: 